In [464]:
import pandas as pd 
import yfinance as yf
import numpy as np
import random
from scipy.optimize import minimize

In [465]:
dataset = pd.read_csv('dataset.csv')

In [466]:
dataset

,Date,stock,adj_close,bbr,pnlog
0,2015-01-02,AAPL,99.945885,58.343195,0.010336
1,2015-01-05,AAPL,97.130241,65.384615,0.014229
2,2015-01-06,AAPL,97.139420,73.206751,0.017950
3,2015-01-07,AAPL,98.501518,67.732123,0.000000
4,2015-01-08,AAPL,102.286186,62.257496,0.005537
...,...,...,...,...,...
20515,2020-06-03,XOM,49.240002,NaN,NaN
20516,2020-06-04,XOM,49.099998,37.168142,-0.011122
20517,2020-06-05,XOM,53.080002,39.331027,-0.017159
20518,2020-06-08,XOM,54.740002,100.000000,0.000000


In [467]:
# dataset = dataset[~dataset['stock'].isin(['JPM', 'KO', 'MCD', 'MSFT', 'PFE', 'XOM'])]

In [468]:
interval = '1d'
start= '2015-01-01'
end= '2020-06-09'

In [469]:
sp500 = yf.download('SPY', start=start, end=end, interval='1d')
sp500

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,206.380005,206.880005,204.179993,205.429993,176.134628,121465900
2015-01-05,204.169998,204.369995,201.350006,201.720001,172.953659,169632600
2015-01-06,202.089996,202.720001,198.860001,199.820007,171.324631,209151400
2015-01-07,201.419998,202.720001,200.880005,202.309998,173.459595,125346700
2015-01-08,204.009995,206.160004,203.990005,205.899994,176.537628,147217800
...,...,...,...,...,...,...
2020-06-02,306.549988,308.130005,305.100006,308.079987,293.203827,74267200
2020-06-03,310.239990,313.220001,309.940002,312.179993,297.105865,92567600
2020-06-04,311.109985,313.000000,309.079987,311.359985,296.325470,75794400


In [470]:
sp500['adjclose'] = sp500['Adj Close']

In [471]:
sp500 = sp500[['adjclose']]


In [472]:
sp500['SP500_return'] = (sp500['adjclose'] / sp500['adjclose'].shift(1) - 1) * 100

/var/folders/hj/n94x31sj08bdmgwjs6ybxbf40000gn/T/ipykernel_8491/2497128406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500['SP500_return'] = (sp500['adjclose'] / sp500['adjclose'].shift(1) - 1) * 100


In [473]:
sp500

,adjclose,SP500_return
Date,,
2015-01-02,176.134628,NaN
2015-01-05,172.953659,-1.805987
2015-01-06,171.324631,-0.941887
2015-01-07,173.459595,1.246151
2015-01-08,176.537628,1.774496
...,...,...
2020-06-02,293.203827,0.828005
2020-06-03,297.105865,1.330828
2020-06-04,296.325470,-0.262666


In [474]:
# Convert 'date' column in df2 to datetime (if it's not already)
dataset['Date'] = pd.to_datetime(dataset['Date'])

# Set 'date' column as the index in df2
dataset.set_index('Date', inplace=True)

# Join the two dataframes
df = sp500.join(dataset, how='inner')  # Use how='left' for left join, 'outer' for outer join, etc.


In [475]:
# Read the file into a DataFrame, skipping the last row
ff = pd.read_csv('F-F_Research_Data_Factors_daily.CSV', skiprows= 4, skipfooter=1, engine='python', index_col=0)



In [476]:
ff['Date'] = ff.index
ff['Date'] = pd.to_datetime(ff.index, format='%Y%m%d')
ff = ff.set_index('Date')


In [477]:
ff

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.24,-0.28,0.009
1926-07-02,0.45,-0.32,-0.08,0.009
1926-07-06,0.17,0.27,-0.35,0.009
1926-07-07,0.09,-0.59,0.03,0.009
1926-07-08,0.21,-0.36,0.15,0.009
...,...,...,...,...
2021-01-25,0.28,-0.07,-0.45,0.000
2021-01-26,-0.32,0.00,0.03,0.000
2021-01-27,-2.53,1.30,1.47,0.000


In [478]:
df = ff.join(df, how='inner') 

In [479]:
df

,Mkt-RF,SMB,HML,RF,adjclose,SP500_return,stock,adj_close,bbr,pnlog
Date,,,,,,,,,,
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,AAPL,99.945885,58.343195,0.010336
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,ABBV,51.718132,50.000000,-0.015882
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,AMZN,308.519989,44.823386,-0.020067
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,DB,28.562273,37.650602,0.004023
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,DIS,87.376266,53.571429,0.008930
...,...,...,...,...,...,...,...,...,...,...
2020-06-08,1.39,0.37,2.15,0.0,307.593781,1.208764,KO,49.849998,100.000000,0.000000
2020-06-08,1.39,0.37,2.15,0.0,307.593781,1.208764,MCD,202.649994,100.000000,0.000000
2020-06-08,1.39,0.37,2.15,0.0,307.593781,1.208764,MSFT,188.360001,55.010020,-0.004798


In [480]:
df

,Mkt-RF,SMB,HML,RF,adjclose,SP500_return,stock,adj_close,bbr,pnlog
Date,,,,,,,,,,
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,AAPL,99.945885,58.343195,0.010336
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,ABBV,51.718132,50.000000,-0.015882
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,AMZN,308.519989,44.823386,-0.020067
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,DB,28.562273,37.650602,0.004023
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,DIS,87.376266,53.571429,0.008930
...,...,...,...,...,...,...,...,...,...,...
2020-06-08,1.39,0.37,2.15,0.0,307.593781,1.208764,KO,49.849998,100.000000,0.000000
2020-06-08,1.39,0.37,2.15,0.0,307.593781,1.208764,MCD,202.649994,100.000000,0.000000
2020-06-08,1.39,0.37,2.15,0.0,307.593781,1.208764,MSFT,188.360001,55.010020,-0.004798


In [481]:
df = df.sort_values(by=['stock']).sort_index()


In [482]:
stock=list(set(df['stock']))
data = []  # list to hold dataframes
for i in range(0,len(stock)):
    stock_data=df[df['stock']==stock[i]]
    data.append(stock_data)


In [483]:
data=pd.concat(data)

In [484]:
data

,Mkt-RF,SMB,HML,RF,adjclose,SP500_return,stock,adj_close,bbr,pnlog
Date,,,,,,,,,,
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,GOOG,523.373108,62.622036,0.007672
2015-01-05,-1.84,0.34,-0.68,0.0,172.953659,-1.805987,GOOG,512.463013,52.854812,-0.002559
2015-01-06,-1.04,-0.78,-0.30,0.0,171.324631,-0.941887,GOOG,500.585632,68.195719,0.017585
2015-01-07,1.19,0.18,-0.64,0.0,173.459595,1.246151,GOOG,499.727997,54.057592,-0.004112
2015-01-08,1.81,-0.12,-0.28,0.0,176.537628,1.774496,GOOG,501.303680,71.653543,0.015942
...,...,...,...,...,...,...,...,...,...,...
2020-06-02,0.81,-0.05,0.45,0.0,293.203827,0.828005,ABBV,91.040001,68.914286,0.012053
2020-06-03,1.42,0.11,2.66,0.0,297.105865,1.330828,ABBV,90.889999,37.832311,-0.027009
2020-06-04,-0.34,-0.24,2.98,0.0,296.325470,-0.262666,ABBV,93.040001,38.028169,-0.033180


In [485]:
data['stock_return'] = (data['adjclose'] / data['adjclose'].shift(1)-1) * 100

In [486]:
data['y'] = data['stock_return'] - (data['RF'] / 100)


In [487]:
data

,Mkt-RF,SMB,HML,RF,adjclose,SP500_return,stock,adj_close,bbr,pnlog,stock_return,y
Date,,,,,,,,,,,,
2015-01-02,-0.12,-0.60,0.08,0.0,176.134628,NaN,GOOG,523.373108,62.622036,0.007672,NaN,NaN
2015-01-05,-1.84,0.34,-0.68,0.0,172.953659,-1.805987,GOOG,512.463013,52.854812,-0.002559,-1.805987,-1.805987
2015-01-06,-1.04,-0.78,-0.30,0.0,171.324631,-0.941887,GOOG,500.585632,68.195719,0.017585,-0.941887,-0.941887
2015-01-07,1.19,0.18,-0.64,0.0,173.459595,1.246151,GOOG,499.727997,54.057592,-0.004112,1.246151,1.246151
2015-01-08,1.81,-0.12,-0.28,0.0,176.537628,1.774496,GOOG,501.303680,71.653543,0.015942,1.774496,1.774496
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-02,0.81,-0.05,0.45,0.0,293.203827,0.828005,ABBV,91.040001,68.914286,0.012053,0.828005,0.828005
2020-06-03,1.42,0.11,2.66,0.0,297.105865,1.330828,ABBV,90.889999,37.832311,-0.027009,1.330828,1.330828
2020-06-04,-0.34,-0.24,2.98,0.0,296.325470,-0.262666,ABBV,93.040001,38.028169,-0.033180,-0.262666,-0.262666


In [488]:
data.dropna(inplace=True)

In [489]:
from sklearn.linear_model import LinearRegression

In [490]:
model = LinearRegression()

# Define the independent variables (factors)
X = data[['Mkt-RF', 'SMB', 'HML']]  # Replace with the actual column names of your factors

# Define the dependent variable
y = data['y']  # Replace with the actual column name of your dependent variable

# Fit the linear regression model
model.fit(X, y)

# Print the coefficients
coefficients = model.coef_
intercept = model.intercept_

print("Intercept:", intercept)
print("Coefficients:", coefficients)




Intercept: 0.0030753603737543878
Coefficients: [ 9.74030538e-01 -1.24971439e-01 -4.94754271e-05]


In [491]:
predicted_values = model.predict(X)

In [492]:
data['FittedValues'] = predicted_values

In [493]:
data

,Mkt-RF,SMB,HML,RF,adjclose,SP500_return,stock,adj_close,bbr,pnlog,stock_return,y,FittedValues
Date,,,,,,,,,,,,,
2015-01-05,-1.84,0.34,-0.68,0.0,172.953659,-1.805987,GOOG,512.463013,52.854812,-0.002559,-1.805987,-1.805987,-1.831597
2015-01-06,-1.04,-0.78,-0.30,0.0,171.324631,-0.941887,GOOG,500.585632,68.195719,0.017585,-0.941887,-0.941887,-0.912424
2015-01-07,1.19,0.18,-0.64,0.0,173.459595,1.246151,GOOG,499.727997,54.057592,-0.004112,1.246151,1.246151,1.139709
2015-01-08,1.81,-0.12,-0.28,0.0,176.537628,1.774496,GOOG,501.303680,71.653543,0.015942,1.774496,1.774496,1.781081
2015-01-09,-0.85,0.01,-0.48,0.0,175.122910,-0.801369,GOOG,494.811493,53.968254,-0.006977,-0.801369,-0.801369,-0.826077
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-01,0.52,-0.02,0.45,0.0,290.796021,0.404196,ABBV,90.699997,44.786730,-0.016372,0.404196,0.404196,0.512048
2020-06-02,0.81,-0.05,0.45,0.0,293.203827,0.828005,ABBV,91.040001,68.914286,0.012053,0.828005,0.828005,0.798266
2020-06-03,1.42,0.11,2.66,0.0,297.105865,1.330828,ABBV,90.889999,37.832311,-0.027009,1.330828,1.330828,1.372320


In [494]:
data['stock'].unique()

array(['GOOG', 'AMZN', 'HAL', 'FB', 'DIS', 'XOM', 'MSFT', 'DB', 'KO',
       'JPM', 'MCD', 'HSBC', 'AAPL', 'PFE', 'ABBV'], dtype=object)

In [495]:
unique_stocks = data['stock'].unique()

# Create a new DataFrame to store the transformed data
transformed_data = pd.DataFrame()

# Iterate over the unique stocks and create subsets of columns
for stock in unique_stocks:
    # Filter data for the current stock
    stock_data = data[data['stock'] == stock]
    
    # Extract the 'adj_close' column for the current stock
    adj_close = stock_data['adj_close']
    
    # Add the 'adj_close' column to the transformed data DataFrame
    transformed_data[stock] = adj_close
    
    # Iterate over the remaining columns and add them to the transformed data DataFrame
    for column in stock_data.columns:
        if column not in ['stock', 'adj_close']:
            transformed_data[column] = stock_data[column]

In [502]:
transformed_data.to_csv('transformed.csv')

In [497]:


# Compute the covariance matrix using the 3-factor returns
covariance_matrix = selected_data['FittedValues'].cov()

# Define the objective function for portfolio optimization
def objective(weights, covariance_matrix):
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    return portfolio_variance

# Define the constraint for portfolio weights summing to 1
def constraint(weights):
    return np.sum(weights) - 1.0

# Define the initial guess for portfolio weights
initial_weights = np.ones(len(selected_stocks)) / len(selected_stocks)

# Define the bounds for portfolio weights (0 <= weight <= 1)
bounds = [(0, 1)] * len(selected_stocks)

# Define the equality constraint for portfolio weights summing to 1
constraint_eq = {'type': 'eq', 'fun': constraint}

# Run the portfolio optimization to find the minimum variance portfolio
result = minimize(objective, initial_weights, args=(covariance_matrix,), method='SLSQP', bounds=bounds, constraints=constraint_eq)

# Get the optimal portfolio weights
optimal_weights = result.x

# Print the optimal weights for each stock
for stock, weight in zip(selected_stocks, optimal_weights):
    print(f"{stock}: {weight}")

# Calculate the portfolio return
portfolio_return = np.sum(selected_data[selected_stocks] * optimal_weights)

# Print the portfolio return
print("Portfolio Return:", portfolio_return)

NameError: name 'selected_data' is not defined

In [ ]:
# begin_date = "2018-01-01"  # "2015-01-01"
# end_date = "2019-12-20"  # "2019-12-20" #"2017-12-31"
# period = f"{begin_date}/{end_date}"


In [ ]:
# import pandas as pd
# import numpy as np
# from scipy.linalg import solve
# import statsmodels.api as sm
# from sklearn.metrics import mean_squared_error
# import matplotlib.pyplot as plt
# import seaborn as sns

# # assuming 'dataset', 'fama_lib', and 'SP500_index' are pandas DataFrame or Series

# stock_prices = dataset['adjusted'][period]
# stock_prices.index = pd.to_datetime(stock_prices.index)

# X = np.log(stock_prices).diff().dropna()

# N = X.shape[1]  # number of stocks
# T = X.shape[0]  # number of days

# F_FamaFrench = fama_lib.loc[X.index] / 100

# f_SP500 = np.log(SP500_index).diff().dropna()

# BBrMkt = dataset['BBr'].rolling(window=period).mean().fillna(method='bfill') / 100
# PNlogMkt = dataset['PNlog'].rolling(window=period).mean().fillna(method='bfill')

# SentIndx = PNlogMkt.loc[X.index]

# FFS = pd.concat([fama_lib.loc[X.index], SentIndx], axis=1).fillna(method='bfill')

# SPS = pd.concat([f_SP500.loc[X.index], SentIndx], axis=1).fillna(method='bfill')

# T_trn = round(0.5 * T)
# X_trn = X.iloc[:T_trn]
# X_tst = X.iloc[T_trn:]
# F_FamaFrench_trn = F_FamaFrench.iloc[:T_trn]
# F_FamaFrench_tst = F_FamaFrench.iloc[T_trn:]
# f_SP500_trn = f_SP500.iloc[:T_trn]
# f_SP500_tst = f_SP500.iloc[T_trn:]
# SentIndx_trn = SentIndx.iloc[:T_trn]
# SentIndx_tst = SentIndx.iloc[T_trn:]
# FFS_trn = FFS.iloc[:T_trn]
# FFS_tst = FFS.iloc[T_trn:]
# SPS_trn = SPS.iloc[:T_trn]
# SPS_tst = SPS.iloc[T_trn:]

# Sigma_SCM = X_trn.cov()

# # 1-factor model SP500
# model = sm.OLS(X_trn, sm.add_constant(f_SP500_trn))
# results = model.fit()
# Gamma = results.params.T
# alpha = Gamma[0]
# beta = Gamma[1:]
# resid = results.resid
# Psi_sp500 = np.cov(resid, rowvar=False)
# Sigma_SP500 = f_SP500_trn.var() * np.outer(beta, beta) + np.diag(np.diag(Psi_sp500))

# # and so on for the other models

# Sigma_true = X_tst.cov()

# errors = {
#     'SCM': np.linalg.norm(Sigma_SCM - Sigma_true, ord='fro'),
#     'SP500': np.linalg.norm(Sigma_SP500 - Sigma_true, ord='fro'),
#     # ...
# }

# # barplot of errors
# plt.bar(errors.keys(), errors.values(), color='aquamarine')
# plt.title('Error in estimation of covariance matrix')
# plt.show()

# ref = np.linalg.norm(Sigma_SCM - Sigma_true, ord='fro')**2
# PRIAL = 100 * (ref - np.array(list(errors.values()))**2) / ref

# # barplot of PRIAL
# plt.bar(errors.keys(), PRIAL, color='bisque')
# plt.title('PRIAL for estimation of covariance matrix')
# plt.show()
